<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Defcon4_TFLite_Mobilenet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# sudo apt -y install libportaudio2
# pip install -q tflite-model-maker
print('Updated: 09/12/2022')

In [ ]:
from __future__ import absolute_import
import os, shutil, tarfile 
from os.path import exists, join
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

from google.colab import drive, files
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

contentPath=os.getcwd()

if exists('/content/sample_data'):
    shutil.rmtree('/content/sample_data')

drive.mount('/content/drive', force_remount=True)

if not exists('BashColors.py'):
    shutil.copy2(src='/content/drive/MyDrive/PythonFiles/BashColors.py',
                 dst=contentPath)
    
if not exists('TarfileFunctions.py'):
    shutil.copy2(src='/content/drive/MyDrive/TarfileFunctions.py',
                 dst=contentPath)
    
if not exists('DataGenerator5.tar.gz'):
    shutil.copy2(src='/content/drive/MyDrive/DataGenerator5.tar.gz',
                 dst=contentPath)
    os.chdir(contentPath)
    tff.extractTarfiles('DataGenerator5.tar.gz')

if not exists('GetTPU.py'):
    shutil.copy2(src='/content/drive/MyDrive/PythonFiles/GetTPU.py',
                 dst=contentPath)
    
if not exists('images.tar.gz'):
    shutil.copy2(src='/content/drive/MyDrive/images.tar.gz',
                 dst=contentPath)
    os.chdir(contentPath)
    tff.extractTarfiles('/content/drive/MyDrive/images.tar.gz')
    
from BashColors import C
from GetTPU import *
from TarfileFunctions import *

image_path=os.path.join(contentPath, 'images')
generatorPath=os.path.join(contentPath, 'DataGenerator')
generatorPath

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'blue'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

from tensorflow_examples.lite.model_maker.core.data_util.image_dataloader import ImageClassifierDataLoader
def plotDatasetImages(dataset:ImageClassifierDataLoader):
    """ """
    plt.figure(figsize=(12,12))
    for i, (image, label) in enumerate(dataset.gen_dataset().unbatch().take(25)):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image.numpy(), cmap=plt.cm.gray)
        plt.xlabel(data.index_to_label[label.numpy()])
    plt.show()

# plotDatasetImages(dataset=train_ds)

In [ ]:
BATCH_SIZE = 16
data = DataLoader.from_folder(filename=generatorPath, shuffle=False)
train_ds, restData = data.split(0.8)
val_ds, test_ds = restData.split(0.5)
print()
print('\classes:', train_ds.num_classes)
print('class names', train_ds.index_to_label)
print(len(train_ds) // BATCH_SIZE, len(val_ds) // BATCH_SIZE)
print(train_ds.size // BATCH_SIZE, val_ds.size // BATCH_SIZE)

In [ ]:
help(image_classifier.ImageClassifier.predict_top_k)


In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [ ]:
from tflite_model_maker import image_classifier
BATCH_SIZE = 16
EPOCHS = 1
DROPOUT_RATE = 0.5
LEARNING_RATE = 0.001

model = image_classifier.create(
    train_data = train_ds,
    validation_data = val_ds,
    model_spec = model_spec.get('mobilenet_v2'),
    learning_rate = LEARNING_RATE,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    dropout_rate = DROPOUT_RATE,
    use_augmentation = False, # default=False
    use_hub_library = True,  # default=True
    momentum = 0.9, # Only used when use_hub_library is True
    shuffle = False,
    train_whole_model = True,
)

In [ ]:
bs='''
model  =  tflite_model_maker.image_classifier.create(
    train_ds, model_spec = 'mobilenet_v2',
    validation_data = val_ds, # default
    batch_size = None, # default None
    epochs = None,  # default None
    steps_per_epoch = None,  # default None
    train_whole_model = None, # default None
    dropout_rate = None,  # default None
    learning_rate = None,  # default None
    momentum = None, # default None
    shuffle = False,
    use_augmentation = False, # default False
    use_hub_library = True, warmup_steps = None,  # default None
    model_dir = None, # default None
    do_train = True
)
'''

In [ ]:
model.ALLOWED_EXPORT_FORMAT

In [ ]:
loss, accuracy = model.evaluate(data=val_ds)

In [ ]:
model.export(export_dir='.',
             tflite_filename='Defcon_MobileNetV2.tflite')

In [ ]:
config = QuantizationConfig.for_float16()

model.export(export_dir='.',
             tflite_filename='Defcon_MobileNetV2_fp16.tflite',
             quantization_config=config)

In [ ]:
# help(DeprecationWarning)

In [ ]:
testData = DataLoader.from_folder(image_path, shuffle=False)
print('\n', type(testData), '\n')

In [ ]:
train_ds, rest_data = data.split(0.5)
validation_data, val_ds = rest_data.split(0.5)

In [ ]:
import tensorflow_hub as hub

module_spec = hub.load_module_spec('https://tfhub.dev/google/imagenet/mobilenet_v2_035_128/feature_vector/2')
height, width = hub.get_expected_image_size(module_spec)
images = ...  # A batch of images with shape [batch_size, height, width, 3].
module = hub.Module(module_spec)
features = module(images)   # A batch with shape [batch_size, num_features].